# Команды Git. Часть 2 

Сегодня мы поговорим о принципах работы следующих команд:
* git merge
* git mergetool
* git reset
* git rebase

# 1. git merge

<i> Команда используется для слияния нескольких историй разработки.</i>

<i> Демонстрировать работу команды <b>git merge</b> будем на примере репозитория со следующей структурой коммитов:</i>

In [1]:
#               master
#                 |
#                 V
# 159f1c9 <---- 702bef8

<i> С каждым коммитом добавлялся файл 1.txt и 2.txt, соответственно. </i>

Рассмотрим несколько ситуаций:

#### 1) Создадим ветку new_branch, добавим файл 3.txt и закоммитим изменения:

In [18]:
#               master <- HEAD
#                 |
#                 V
# 159f1c9 <---- 702bef8 <---- 26e644d
#                                ^
#                                |
#                           new_branch

In [20]:
! git merge new_branch

Updating 702bef8..26e644d
Fast-forward
 3.txt | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 3.txt


In [6]:
#                           new_branch, master <- HEAD
#                                |
#                                V
# 159f1c9 <---- 702bef8 <---- 26e644d

Было выполнено быстрое слияние. Если мы хотим следующий вариант ветвления

In [ ]:
#                           master <- HEAD
#                                |
#                                V
# 159f1c9 <---- 702bef8 <---- merge_commit 
#                  ^              
#                  |              
#               26e644d
#                  ^
#                  |
#              new_branch

то можно воспользоваться флагом --no-ff.

Если нужно слить две ветки, но не создавать коммит сразу, используется флаг --no-commit. Изменения будут находиться в индексе.

#### 2) Пусть теперь ветка new_branch ссылается на 159f1c9, добавим в файл 1.txt новую строку и закоммитим изменения:

In [7]:
#                             master <- HEAD
#                                |
#                                V
# 159f1c9 <---- 702bef8 <---- 26e644d
#    ^
#    |
# 679d9f4 
#    ^
#    |
# new_branch

In [21]:
! git merge new_branch

Merge made by the 'recursive' strategy.
 1.txt | 3 ++-
 1 file changed, 2 insertions(+), 1 deletion(-)


In [10]:
#                                         master <- HEAD
#                                              |
#                                              V
# 159f1c9 <---- 702bef8 <---- 26e644d <---- 73cba1b
#    ^
#    |
# 679d9f4 
#    ^
#    |
# new_branch

Стратегии слияния бывают следующего типа:
* resolve,
* recursive,
* ours, 
* octopus,
* subtree.

Выбор стратегии производится с помощью ключа -s.

В данном примере resolve не отличается от recursive. Различия будут, например, вот в таком случае слияния веток branch1 и branch2:

In [2]:
# A - B - E - F <- branch1
#  \     |        
#   C - D - G <- branch2

#### 3) По-разному изменим 1.txt в master и new_branch:

In [27]:
#                         master <- HEAD                
#                            |
#                            V
# 159f1c9 <---- ... <---- 0c8347a  1.txt (v2)            v1: I want to change file
#    ^                                                   v2: I like conflicts
#    |
# 679d9f4
#    ^
#    |
# f3bbc56   1.txt (v1)
#    ^
#    |
# new_branch

In [22]:
! git merge new_branch

Auto-merging 1.txt
CONFLICT (content): Merge conflict in 1.txt
Automatic merge failed; fix conflicts and then commit the result.


In [26]:
with open("1.txt", 'r') as f:
    print(f.read())

<<<<<<< HEAD
I like conflicts
I want to change that file
>>>>>>> new_branch



Если исправлять конфликты в данный момент не хочется, то можно выполнить команду

In [1]:
# git merge --abort

# 2. git mergetool

<i> Позволяет в графическом режиме разрешать конфликты. </i>

* Base: общий предок
* Local: версия в текущей ветке (master)
* Remote: версия в ветка, откуда хотим влить изменения (new_branch)

<img src='./pic/001.png' width="900">

После разрешения всех конфликтов делаем коммит:

In [29]:
#                         master <- HEAD                
#                            |
#                            V
# 159f1c9 <---- ... <---- b1e55c1  1.txt (v3)      
#    ^                                          
#    |                                                  
# 679d9f4
#    ^
#    |
# f3bbc56   1.txt (v1)
#    ^
#    |
# new_branch

# 3. git reset

<i> Позволяет отменять нежелательные изменения. </i>

Рассмотрим следующий пример:

<img src='./pic/002.png' width="500">

<b> Вариант 1: </b>

In [ ]:
# git reset -- soft 9e5e6a4

или

In [35]:
# git reset -- soft HEAD~

<img src='./pic/003.png' width="500">

<b> Вариант 2: </b>

In [36]:
# git reset --mixed HEAD~

или

In [37]:
# git reset HEAD~

<img src='./pic/004.png' width="500">

<b> Вариант 3: </b>

In [40]:
# git reset --hard HEAD~

<img src='./pic/005.png' width="500">

Важно отметить, что только указание этого флага (--hard) делает команду reset опасной, это один из немногих случаев, когда Git действительно удаляет данные. Все остальные вызовы reset легко отменить

Итого, команда git reset выполняет 3 операции:
* перемещает то, на что указывает HEAD <b>(--soft)</b>,
* обновляет индекс <b>(--mixed)</b>,
* обновляет рабочий каталог <b>(--hard)</b>.

Команде git reset также можно передать путь. Например, команда

In [3]:
# git reset file.txt

отработает следующим образом:
* не меняет положение HEAD,
* индексирует file.txt в соответсвии с HEAD
Команда

In [ ]:
# git reset eb43bf file.txt

обновит индекс в соответствии с первым коммитом.

<img src='./pic/008.png' width="800">

# 4. git rebase

<i> Ещё один способ объединения веток.</i>

Иллюстрации взяты из статьи https://habrahabr.ru/post/161009/

<img src='./pic/009.png' width="500">

In [8]:
# git rebase master

<img src='./pic/010.png' width="500">

#### Шаг 1.

<img src='./pic/012.png' width="500">

#### Шаг 2.

<img src='./pic/013.png' width="500">

Возник конфликт. Два варианта развития событий:
* отмена процесса (git rebase --abort)
* решение конфликтов и продолжение процесса (git rebase --continue)

#### Шаг 3.

<img src='./pic/014.png' width="500">

# 4*. git rebase в реальном проекте

#### Объединение с merge:

<img src='./pic/015.png' width="700">

#### Объединение с rebase:

<img src='./pic/016.png' width="700">